<a href="https://colab.research.google.com/github/AnIsAsPe/LDA-TopicModeling_python/blob/main/Notebooks/LDA_con_sklearn_noticias_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar y cargar bibliotecas

In [1]:
!pip install pyLDAvis  #biblioteca que extrae información de un modelo LDA para obtener una visualización interactiva

     |████████████████████████████████| 1.7 MB 7.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=b757ec71deac42709b5491a6ae0561c3debd0a8c17949102db7fe7b3dcaf83ec
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [2]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import matplotlib.pyplot as plt 
import seaborn as sns  

nltk.download('stopwords')
nltk.download('wordnet')  #WordNetLemmatizer
nltk.download('punkt')


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Lectura y exploración de datos

En colab, al instalar la librería pyLDAvis causa un conflicto con Pandas. Si esto se refleja al correr la siguiente linea es necesario reiniciar el entorno de ejecución.

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Datos/ABC News/abcnews-date-text.csv', parse_dates=['publish_date'] )
print(df.shape)
df.head()

(1226258, 2)


,publish_date,headline_text
0,2003-02-19,aba decides against community broadcasting lic...
1,2003-02-19,act fire witnesses must be aware of defamation
2,2003-02-19,a g calls for infrastructure protection summit
3,2003-02-19,air nz staff in aust strike for pay rise
4,2003-02-19,air nz strike to affect australian travellers


In [4]:
df.publish_date.min(), df.publish_date.max()

(Timestamp('2003-02-19 00:00:00'), Timestamp('2020-12-31 00:00:00'))

In [5]:
# promedio de noticias por día
len(df)/(df.publish_date.max()-df.publish_date.min()).days


187.93226053639847

In [6]:

filtro = (df['publish_date']>= '2019-01-01')&(df['publish_date']<= '2019-12-31')
df = df[filtro]
df = df.reset_index(drop=True )
len(df)

34060

# Vectorización de textos

In [7]:
def preprocesar(texto):
  #convierte a minúsculas
  texto = (texto).lower()

  #elimina stopwords
  stop = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
  texto = stop.sub('', texto) 

  #Quitar puntuación y números
  texto = re.sub('[^ña-z]+', ' ', texto)

  #lematizar y quedarnos con palabras que tengan más de tres caracteres
  lemmatizer = WordNetLemmatizer()
  texto = texto.split()
  texto = ' '.join([lemmatizer.lemmatize(i) for i in texto if len(i)>2])
  
  return(texto)

In [8]:
df['headline_pp']=df['headline_text'].apply(preprocesar)
df.head()

,publish_date,headline_text,headline_pp
0,2019-01-01,after expo ministers approved an artificial is...,expo minister approved artificial island
1,2019-01-01,alyssa healy named worlds best womens t20 play...,alyssa healy named world best woman player
2,2019-01-01,australia called a free rider on tackling clim...,australia called free rider tackling climate c...
3,2019-01-01,australia still has no us ambassador,australia still ambassador
4,2019-01-01,bangladesh ruling coalition declared winner of...,bangladesh ruling coalition declared winner di...


In [9]:
vectorizer = CountVectorizer(min_df=3, ngram_range=(1,2))
BOW = vectorizer.fit_transform(df['headline_pp'])
BOW.shape

(34060, 15310)

In [10]:
vocabulario = vectorizer.get_feature_names_out()
len(vocabulario)

15310

In [11]:
list(vocabulario)[0:30]

['aaron',
 'aaron elli',
 'aaron finch',
 'ab',
 'ab data',
 'abalone',
 'abandon',
 'abandoned',
 'abares',
 'abares crop',
 'abattoir',
 'abbey',
 'abbey road',
 'abbot',
 'abbot point',
 'abbott',
 'abbott zali',
 'abbotts',
 'abc',
 'abc chair',
 'abc investigation',
 'abc managing',
 'abc news',
 'abc office',
 'abc presenter',
 'abc radio',
 'abc raid',
 'abc report',
 'abc reporter',
 'abc weather']

# Entrenamiento del modelo

In [12]:
lda_model=LatentDirichletAllocation(n_components=5,learning_method='online',random_state=42,max_iter=50) 

In [13]:
%%time
lda_model.fit(BOW) # entrena el modelo y obtienela matriz documento-topico

CPU times: user 3min 14s, sys: 1.13 s, total: 3min 15s
Wall time: 3min 16s


LatentDirichletAllocation(learning_method='online', max_iter=50, n_components=5,
                          random_state=42)

### Distribución de temas en cada noticia  (O)

In [14]:
doc_top = pd.DataFrame(lda_model.transform(BOW))
print(doc_top.shape)
doc_top.head()

(34060, 5)


,0,1,2,3,4
0,0.033335,0.366004,0.033555,0.033335,0.533772
1,0.020001,0.020654,0.020050,0.319527,0.619768
2,0.699813,0.140241,0.119797,0.020097,0.020052
3,0.050313,0.050580,0.050000,0.798849,0.050258
4,0.025001,0.025001,0.025308,0.776513,0.148176


In [15]:
doc_top.sum(axis=1)

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
34055    1.0
34056    1.0
34057    1.0
34058    1.0
34059    1.0
Length: 34060, dtype: float64

In [16]:
pd.merge(df, doc_top, left_index=True, right_index=True)

,publish_date,headline_text,headline_pp,0,1,2,3,4
0,2019-01-01,after expo ministers approved an artificial is...,expo minister approved artificial island,0.033335,0.366004,0.033555,0.033335,0.533772
1,2019-01-01,alyssa healy named worlds best womens t20 play...,alyssa healy named world best woman player,0.020001,0.020654,0.020050,0.319527,0.619768
2,2019-01-01,australia called a free rider on tackling clim...,australia called free rider tackling climate c...,0.699813,0.140241,0.119797,0.020097,0.020052
3,2019-01-01,australia still has no us ambassador,australia still ambassador,0.050313,0.050580,0.050000,0.798849,0.050258
4,2019-01-01,bangladesh ruling coalition declared winner of...,bangladesh ruling coalition declared winner di...,0.025001,0.025001,0.025308,0.776513,0.148176
...,...,...,...,...,...,...,...,...
34055,2019-12-31,vision of flames approaching corryong in victoria,vision flame approaching corryong victoria,0.239760,0.040001,0.440168,0.040001,0.240071
34056,2019-12-31,wa police and government backflip on drug amne...,police government backflip drug amnesty bin,0.157379,0.274911,0.025073,0.517528,0.025110
34057,2019-12-31,we have fears for their safety: victorian premier,fear safety victorian premier,0.033334,0.034067,0.199996,0.366604,0.366000
34058,2019-12-31,when do the 20s start,start,0.100000,0.599999,0.100000,0.100000,0.100000


## Distribución de palabras en cada tema ($\mu$)

In [17]:
μs = pd.DataFrame(lda_model.exp_dirichlet_component_,
                         columns=vocabulario)
print(μs.shape)
μs.head()

(5, 15310)


,aaron,aaron elli,aaron finch,ab,ab data,abalone,abandon,abandoned,abares,abares crop,...,zipline,zlatan,zoe,zombie,zone,zone change,zoo,zoom,zuckerberg,zverev
0,1.165882e-07,1.142381e-07,1.041173e-07,1.128111e-07,1.050417e-07,1.074401e-07,1.099981e-07,1.670480e-04,1.401724e-04,5.769569e-05,...,1.068557e-07,1.133271e-07,1.143372e-07,2.774461e-04,4.424139e-04,4.825280e-05,1.165312e-07,1.053780e-07,1.062255e-07,1.155185e-07
1,1.008305e-07,1.010258e-07,1.017063e-07,2.004628e-04,1.018411e-07,1.080892e-07,2.389434e-04,1.010924e-07,9.601201e-08,9.617893e-08,...,1.042736e-07,9.991715e-08,5.576734e-05,1.208930e-07,1.122073e-07,9.578523e-08,1.080009e-07,4.933455e-05,1.132896e-04,1.536240e-05
2,1.004140e-07,9.837552e-08,9.873605e-08,1.025169e-07,9.923577e-08,7.615516e-05,1.130018e-07,1.135953e-04,1.010884e-07,1.029202e-07,...,1.010977e-07,6.006067e-05,1.097432e-07,1.032753e-07,1.056264e-07,9.775623e-08,1.093966e-07,1.008333e-07,9.784042e-08,9.738872e-08
3,2.768259e-04,7.213952e-05,6.643189e-05,1.062265e-07,9.763503e-08,1.062207e-07,1.087872e-07,1.198866e-07,9.818551e-08,9.764118e-08,...,1.033816e-07,1.026995e-07,1.017436e-07,1.030692e-07,1.069405e-07,9.726929e-08,4.039032e-04,1.099081e-07,9.939553e-08,9.784671e-08
4,1.073122e-07,1.033457e-07,1.072838e-07,1.976160e-07,5.367641e-05,1.078152e-07,1.119844e-07,1.115558e-07,1.042809e-07,1.020046e-07,...,5.731894e-05,1.020243e-07,1.403873e-07,1.174925e-07,1.148552e-07,1.082318e-07,1.067244e-07,1.034218e-07,1.058423e-07,2.502929e-05


In [18]:
μs.sum(axis=1)

0    0.917485
1    0.925248
2    0.923990
3    0.924217
4    0.920748
dtype: float64

# Visualización del modelo

In [19]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_model, BOW, vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.253028  0.165884       1        1  20.844459
3     -0.219695  0.048290       2        1  20.459288
2     -0.219270  0.047527       3        1  20.297888
4      0.085694 -0.363630       4        1  19.313593
0      0.100243  0.101929       5        1  19.084771, topic_info=             Term        Freq        Total Category  logprob  loglift
986    australian  853.000000   853.000000  Default  30.0000  30.0000
14070       trump  727.000000   727.000000  Default  29.0000  29.0000
4509     election  659.000000   659.000000  Default  28.0000  28.0000
5238         fire  623.000000   623.000000  Default  27.0000  27.0000
9389         news  599.000000   599.000000  Default  26.0000  26.0000
...           ...         ...          ...      ...      ...      ...
11921         say  271.829099   810.829760   Topic5  -5.1252   0.5634
1107         back  177.269374   291.144039   Topic5  -5.5527   1.1601
13289      sydney  206.170565   635.980211   Topic5  -5.4017   0.5298
935     australia  199.003061  1077.914311   Topic5  -5.4370  -0.0332
10351      police  173.088399   994.758626   Topic5  -5.5766  -0.0924

[217 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
18         4  0.997444        abc
22         4  0.998243   abc news
64         1  0.997606      abuse
103        2  0.998667    accused
245        5  0.996506        afl
...      ...       ...        ...
15098      3  0.994951     worker
15116      1  0.867518      world
15116      4  0.130816      world
15122      1  0.993856  world cup
15197      3  0.998641       year

[211 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 5, 1])

# Guardamos modelo

In [20]:
import pickle
path = '/content/drive/MyDrive/Modelos/modelosLDA/LDA_sklearn/'
tuple_models = (lda_model, BOW, vectorizer)
pickle.dump(tuple_models, open (path + "tuple_model_news2019.pkl", 'wb'))

# Leemos el guardado modelo

In [21]:
import pickle
path = '/content/drive/MyDrive/Modelos/modelosLDA/LDA_sklearn/'
lda_model, BOW, vectorizer = pickle.load(open(path + "tuple_model_news2019.pkl", 'rb'))


# Entrenamiento del modelo 2

In [22]:
lda_model2=LatentDirichletAllocation(n_components=4,learning_method='online',random_state=42,max_iter=50) 

In [23]:
%%time
lda_model2.fit(BOW) # entrena el modelo y obtienela matriz documento-topico

CPU times: user 3min 9s, sys: 1.28 s, total: 3min 10s
Wall time: 3min 10s


LatentDirichletAllocation(learning_method='online', max_iter=50, n_components=4,
                          random_state=42)

### Distribución de temas en cada noticia  (O)

In [24]:
doc_top = pd.DataFrame(lda_model2.transform(BOW))
print(doc_top.shape)
doc_top.head()

(34060, 4)


,0,1,2,3
0,0.041671,0.874987,0.041671,0.041671
1,0.025560,0.490073,0.025004,0.459363
2,0.805412,0.026529,0.141892,0.026167
3,0.062763,0.064329,0.062501,0.810407
4,0.031256,0.354974,0.033360,0.580410


In [25]:
doc_top.sum(axis=1)

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
34055    1.0
34056    1.0
34057    1.0
34058    1.0
34059    1.0
Length: 34060, dtype: float64

In [26]:
pd.merge(df, doc_top, left_index=True, right_index=True)

,publish_date,headline_text,headline_pp,0,1,2,3
0,2019-01-01,after expo ministers approved an artificial is...,expo minister approved artificial island,0.041671,0.874987,0.041671,0.041671
1,2019-01-01,alyssa healy named worlds best womens t20 play...,alyssa healy named world best woman player,0.025560,0.490073,0.025004,0.459363
2,2019-01-01,australia called a free rider on tackling clim...,australia called free rider tackling climate c...,0.805412,0.026529,0.141892,0.026167
3,2019-01-01,australia still has no us ambassador,australia still ambassador,0.062763,0.064329,0.062501,0.810407
4,2019-01-01,bangladesh ruling coalition declared winner of...,bangladesh ruling coalition declared winner di...,0.031256,0.354974,0.033360,0.580410
...,...,...,...,...,...,...,...
34055,2019-12-31,vision of flames approaching corryong in victoria,vision flame approaching corryong victoria,0.401907,0.050004,0.279421,0.268668
34056,2019-12-31,wa police and government backflip on drug amne...,police government backflip drug amnesty bin,0.032203,0.656093,0.031252,0.280452
34057,2019-12-31,we have fears for their safety: victorian premier,fear safety victorian premier,0.041667,0.874828,0.041668,0.041837
34058,2019-12-31,when do the 20s start,start,0.125002,0.624994,0.125002,0.125002


## Distribución de palabras en cada tema ($\mu$)

In [27]:
μs = pd.DataFrame(lda_model2.exp_dirichlet_component_,
                         columns=vocabulario)
print(μs.shape)
μs.head()

(4, 15310)


,aaron,aaron elli,aaron finch,ab,ab data,abalone,abandon,abandoned,abares,abares crop,...,zipline,zlatan,zoe,zombie,zone,zone change,zoo,zoom,zuckerberg,zverev
0,2.956196e-07,2.964650e-07,2.256042e-07,2.332032e-07,2.447474e-07,2.558697e-07,1.897022e-04,2.187125e-04,1.043556e-04,4.332911e-05,...,3.907076e-07,2.464007e-07,4.523316e-05,1.495830e-04,3.278381e-04,3.640268e-05,2.555242e-07,2.444664e-07,2.259424e-07,2.287317e-07
1,2.469087e-07,2.297260e-07,2.741597e-07,1.624298e-04,4.098880e-05,2.604014e-07,2.241632e-06,2.455758e-07,2.336722e-07,2.291994e-07,...,4.262101e-05,2.554289e-07,3.320083e-07,5.291372e-05,3.783016e-07,2.395008e-07,3.211833e-04,4.052326e-05,4.107143e-07,2.579677e-07
2,2.589788e-07,2.743651e-07,2.455775e-07,2.954051e-07,2.630407e-07,6.674072e-05,3.720522e-07,2.754605e-07,2.834117e-07,2.770335e-07,...,4.831776e-07,5.283615e-05,3.726300e-07,2.773253e-07,3.197514e-07,2.469620e-07,3.187326e-07,2.446160e-07,1.002482e-04,3.907181e-05
3,2.261288e-04,5.926520e-05,5.478419e-05,2.729401e-07,2.531924e-07,2.472064e-07,2.850546e-07,2.969760e-07,2.568277e-07,2.465316e-07,...,2.705113e-07,2.282317e-07,2.794262e-07,2.639823e-07,2.653619e-07,2.451719e-07,2.587686e-07,2.282723e-07,3.101984e-07,2.332814e-07


In [28]:
μs.sum(axis=1)

0    0.927463
1    0.927860
2    0.922533
3    0.925351
dtype: float64

# Visualización del modelo 2

In [30]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_model2, BOW, vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.036360 -0.316707       1        1  25.819579
0     -0.247771  0.210764       2        1  25.543170
3     -0.032441 -0.020535       3        1  25.114143
2      0.316572  0.126478       4        1  23.523108, topic_info=             Term        Freq       Total Category  logprob  loglift
986    australian  857.000000  857.000000  Default  30.0000  30.0000
14070       trump  725.000000  725.000000  Default  29.0000  29.0000
5238         fire  626.000000  626.000000  Default  28.0000  28.0000
4509     election  657.000000  657.000000  Default  27.0000  27.0000
10351      police  992.000000  992.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
15197        year  316.518446  590.276987   Topic4  -5.1821   0.8240
3730          day  235.029634  438.732585   Topic4  -5.4797   0.8230
9389         news  246.905775  602.088219   Topic4  -5.4304   0.5558
4982       family  210.181533  371.003105   Topic4  -5.5915   0.8789
11921         say  239.553098  811.203289   Topic4  -5.4607   0.2274

[179 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
18         2  0.997445        abc
22         2  0.998244   abc news
103        3  0.998200    accused
245        2  0.996611        afl
281        4  0.994384       aged
...      ...       ...        ...
15116      1  0.999359      world
15122      1  0.996014  world cup
15197      1  0.142306       year
15197      3  0.321883       year
15197      4  0.537036       year

[164 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3])